In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

from ibm_botocore.client import Config
import ibm_boto3
import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F


import numpy as np
import pandas as pd
import json
import pyspark.sql
from pyspark.sql.functions import col, skewness, kurtosis
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import when
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from datetime import date, timedelta, datetime
import time

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210210211421-0000
KERNEL_ID = 8f0338a9-dc91-4cab-9963-b666ae82b8f5


In [2]:
from zipfile import ZipFile

credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-3c1861d6-5e36-4bd6-9a2c-193573560cbf',
    'IBM_API_KEY_ID': 'p5ub7Zlafd2TotZeptmWjdN0MbfseCEWl54M7UMVfKtO',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'movielensdataanalyticsinsparkandp-donotdelete-pr-7pdjrbcote4rbl',
    'FILE': ['ratings.dat.zip', 'movies.dat','users.dat']
}

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE'][0],Filename=credentials['FILE'][0])
cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE'][1],Filename=credentials['FILE'][1])
cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE'][2],Filename=credentials['FILE'][2])

with ZipFile(credentials['FILE'][0],'r') as zipObj:
    zipObj.extractall()

In [5]:
import ibmos2spark

credentials2 = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-3c1861d6-5e36-4bd6-9a2c-193573560cbf',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'p5ub7Zlafd2TotZeptmWjdN0MbfseCEWl54M7UMVfKtO'
}

configuration_name = 'os_54ba3cbe01c144af917f4e0e63fb0d9c_configs'
cos2 = ibmos2spark.CloudObjectStorage(sc, credentials2, configuration_name, 'bluemix_cos')


In [6]:

spark.read.text("movies.dat").createOrReplaceTempView("movies_staging")
spark.read.text("ratings.dat").createOrReplaceTempView("ratings_staging")
spark.read.text("users.dat").createOrReplaceTempView("users_staging")

## Create a database to store the tables
spark.sql("drop database if exists sparkdatalake cascade")
spark.sql("create database sparkdatalake");


## Make appropriate schemas for them
## movies
movies_df = spark.sql(""" Select 
split(value,'::')[0] as movieid,
split(value,'::')[1] as title,
substring(split(value,'::')[1],length(split(value,'::')[1])-4,4) as year,
split(value,'::')[2] as genre 
from movies_staging """).write.parquet(cos2.url('moviesdataframe.parquet', 'movielensdataanalyticsinsparkandp-donotdelete-pr-7pdjrbcote4rbl'))

## users
spark.sql(""" Select
split(value,'::')[0] as userid,
split(value,'::')[1] as gender,
split(value,'::')[2] as age,
split(value,'::')[3] as occupation,
split(value,'::')[4] as zipcode
from users_staging """).write.parquet(cos2.url('usersdataframe.parquet', 'movielensdataanalyticsinsparkandp-donotdelete-pr-7pdjrbcote4rbl'))

## ratings
spark.sql(""" Select
split(value,'::')[0] as userid,
split(value,'::')[1] as movieid,
split(value,'::')[2] as rating,
split(value,'::')[3] as timestamp
from ratings_staging """).write.parquet(cos2.url('ratingsdataframe.parquet', 'movielensdataanalyticsinsparkandp-donotdelete-pr-7pdjrbcote4rbl'))


In [9]:
dfreadback = spark.read.parquet(cos.url('moviesdataframe.parquet', 'movielensdataanalyticsinsparkandp-donotdelete-pr-7pdjrbcote4rbl'))
dfreadback.take(4)

[Row(movieid='1', title='Toy Story (1995)', year='1995', genre="Animation|Children's|Comedy"),
 Row(movieid='2', title='Jumanji (1995)', year='1995', genre="Adventure|Children's|Fantasy"),
 Row(movieid='3', title='Grumpier Old Men (1995)', year='1995', genre='Comedy|Romance'),
 Row(movieid='4', title='Waiting to Exhale (1995)', year='1995', genre='Comedy|Drama')]